In [10]:
import pandas as pd
import matplotlib.pyplot as plt
import os
from thefuzz import fuzz, process
import missingno as msno

# Matched counties


In [4]:
matched_counties = pd.read_csv("c:/Users/Zayan/Documents/code/personal_repos/salmonella/Data/processed_data/matched_counties.csv")

In [5]:
matched_counties.head()

,Unnamed: 0,Census,Matched SVI County,Matched PulseNet County,Matched MMG County
0,0,"Autauga County, Alabama",Autauga,COUNTY,"Autauga County, Alabama"
1,1,"Baldwin County, Alabama",Baldwin,Baldwin,"Baldwin County, Alabama"
2,2,"Barbour County, Alabama",Barbour,COUNTY,"Barbour County, Alabama"
3,3,"Bibb County, Alabama",Bibb,Bibb,"Bibb County, Alabama"
4,4,"Blount County, Alabama",Blount,Blount,"Blount County, Alabama"


In [6]:
# Import PulseNet data
pulsenet_path = r"C:\Users\Zayan\Documents\code\personal_repos\salmonella\Data\raw_data\PulseNet data\Export_2020-2023.xlsx"
pulsenet_data = pd.read_excel(pulsenet_path, sheet_name="2020-2023")

# Import Map the Meal Gap (MMG) data
mmg_path = r"c:/Users/Zayan/Documents/code/personal_repos/salmonella/Data/raw_data/MMG/MMG2022_2020-2019Data_ToShare.xlsx"
mmg_data = pd.read_excel(mmg_path, sheet_name='County')

# Import Social Vulnerability Index (SVI) data
svi_path = r"C:\Users\Zayan\Documents\code\personal_repos\salmonella\Data\raw_data\social_vulnerability_index\SVI_2020_US_county.csv"
svi_data = pd.read_csv(svi_path)

# Display the first few rows of each dataset
print("PulseNet Data:")
print(pulsenet_data.head())
print("\nMap the Meal Gap Data:")
print(mmg_data.head())
print("\nSocial Vulnerability Index Data:")
print(svi_data.head())



PulseNet Data:
                Key AntigenForm  AntigenForm_wgs           IsolatDate LabID  \
0  FL___JBS19003284         NaN  I 7:e,h:e,n,z15  2019-11-21 00:00:00    FL   
1  FL___JBS19003283         NaN        I 9:g,m:-  2019-11-25 00:00:00    FL   
2  FL___JBS19003285         NaN        I 4:i:1,2  2019-11-29 00:00:00    FL   
3  FL___JBS19003286         NaN      I 4:e,h:1,2  2019-11-30 00:00:00    FL   
4     MA___19EN7649         NaN        I 7:r:1,5  2019-12-16 00:00:00    MA   

  Outbreak  PatientAgeDays PatientAgeMonths PatientAgeYears PatientSex  ...  \
0      NaN             NaN              NaN              19     FEMALE  ...   
1      NaN             NaN              NaN               4       MALE  ...   
2      NaN             NaN                2             NaN       MALE  ...   
3      NaN             NaN                4             NaN       MALE  ...   
4      NaN             NaN              NaN              26       MALE  ...   

  Serotype_wgs   SourceCity SourceC

In [7]:
pulsenet_data.loc[pulsenet_data['SourceCounty'] == 'COUNTY']


,Key,AntigenForm,AntigenForm_wgs,IsolatDate,LabID,Outbreak,PatientAgeDays,PatientAgeMonths,PatientAgeYears,PatientSex,...,Serotype_wgs,SourceCity,SourceCountry,SourceCounty,SourceSite,SourceState,SourceType,TypeDetails,WGS_id,Unnamed: 21
35523,SC___SALM21-0028,NaN,"I 8:e,h:1,2",2021-01-05 00:00:00,SC,NaN,NaN,NaN,14,FEMALE,...,Newport,NaN,USA,COUNTY,Urine,SC,Human,NaN,PNUSAS190864,NaN


In [2]:
pulsenet_data.loc[pulsenet_data['SourceCounty'] == 'COUNTY']


NameError: name 'pulsenet_data' is not defined

Verify that counties marked as 'COUNTY' in matched_counties['Matched PulseNet County'] 
are indeed not present in the PulseNet data

In [9]:
# Function to find similar county names in PulseNet data
def find_similar_counties(df, county_name, similarity_threshold=80):
    matches = process.extract(county_name, df['SourceCounty'], limit=None)
    return [match[0] for match in matches if match[1] >= similarity_threshold]

# Filter matched_counties for rows where 'Matched PulseNet County' is 'COUNTY'
counties_to_check = matched_counties['Census'][((matched_counties['Matched PulseNet County'] == 'COUNTY') | (matched_counties['Matched PulseNet County'] == ''))]

# Verify each county
for index, row in counties_to_check.iterrows():
    census_county = row['Census Matched SVI County']
    similar_counties = find_similar_counties(pulsenet_data, census_county)
    
    if similar_counties:
        print(f"Warning: {census_county} marked as 'COUNTY' but similar names found in PulseNet: {similar_counties}")
    else:
        print(f"Verified: {census_county} not found in PulseNet data")

# Count and display the number of verified and unverified counties
verified_count = counties_to_check.shape[0] - sum(1 for _ in counties_to_check.iterrows() if find_similar_counties(pulsenet_data, _[1]['Census Matched SVI County']))
unverified_count = counties_to_check.shape[0] - verified_count

print(f"\nSummary:")
print(f"Total counties checked: {counties_to_check.shape[0]}")
print(f"Verified (not found in PulseNet): {verified_count}")
print(f"Unverified (similar names found in PulseNet): {unverified_count}")

AttributeError: 'Series' object has no attribute 'iterrows'

In [15]:
def find_specific_counties(df, county_name, similarity_threshold=80):
    matches = process.extract(county_name, df['SourceCounty'], limit=None)
    return [match[0] for match in matches if match[1] >= similarity_threshold]

find_specific_counties(pulsenet_data, 'kusilvak')
find_specific_counties(pulsenet_data, 'autauga')



['Watauga', 'Watauga', 'WATAUGA', 'WATAUGA', 'WATAUGA', 'WATAUGA']